# 🎨 Anima LoRA Training (diffusion-pipe / Colab T4)

> **Anima** — 2B parameter anime-focused text-to-image model (Cosmos-Predict2 기반, CircleStone Labs)
>
> [diffusion-pipe](https://github.com/tdrussell/diffusion-pipe)를 이용한 DeepSpeed 기반 LoRA 학습 노트북
>
> **환경:** Colab T4 (16GB VRAM) / bf16 / Prodigy / rank 16
>
> 주요 하이퍼파라미터는 셀 4-1, 4-2의 `@param` 폼에서 조절 가능합니다.

### 고정 설정 (변경 비권장)
| 항목 | 값 | 이유 |
|---|---|---|
| dtype | `bfloat16` | PyTorch 2.9+에서 T4 bf16 지원, fp16은 NaN 위험 |
| pipeline_stages | 1 | LoRA 학습에서는 1 고정 |
| activation_checkpointing | true | T4 VRAM 필수 |

## 1. 환경 설정 및 의존성 설치

In [ ]:
#@title 1-1. GPU 확인 및 CUDA 메모리 설정
import torch, subprocess

# GPU 확인
gpu_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else "NO GPU"
gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1024**3 if torch.cuda.is_available() else 0
print(f"🖥️  GPU: {gpu_name}")
print(f"💾 VRAM: {gpu_mem:.1f} GB")
print(f"🔧 CUDA: {torch.version.cuda}")
print(f"🐍 PyTorch: {torch.__version__}")

# bf16 지원 여부 확인
bf16_support = torch.cuda.is_bf16_supported()
print(f"📋 BF16 지원: {bf16_support}")
if bf16_support:
    print("   → BF16 모드로 학습합니다")
else:
    print("   ⚠️ BF16 미지원 — TOML에서 dtype을 float16으로 수정 필요")

# CUDA 메모리 단편화 방지
import os
os.environ['PYTORCH_ALLOC_CONF'] = 'expandable_segments:True'

In [ ]:
#@title 1-2. diffusion-pipe 클론 및 의존성 설치
import os

# diffusion-pipe 클론 (ComfyUI 등 submodule 포함 필수!)
if not os.path.exists('/content/diffusion-pipe'):
    !git clone --recurse-submodules https://github.com/tdrussell/diffusion-pipe.git /content/diffusion-pipe
else:
    print("diffusion-pipe already cloned")

os.chdir('/content/diffusion-pipe')

# submodule이 비어있으면 초기화 (comfy 모듈 누락 방지)
!git submodule init && git submodule update

# 핵심 의존성 설치
# transformers<5.0 고정: v5+에서 batch_encode_plus 제거됨 (Anima tokenizer 호환 이슈)
# deepspeed: diffusion-pipe의 핵심 학습 프레임워크
!pip install -q deepspeed==0.18.4
!pip install -q "transformers>=4.45,<5.0" "diffusers>=0.35.1" accelerate peft safetensors
!pip install -q bitsandbytes torch-optimi pytorch-optimizer
!pip install -q toml datasets pillow sentencepiece protobuf
!pip install -q tensorboard tqdm einops loguru
!pip install -q huggingface_hub

!pip install -q imageio[ffmpeg] av omegaconf
print("✅ 의존성 설치 완료")


## 2. 모델 다운로드 (HuggingFace → Colab)

Anima 모델 3개 파일을 HF에서 직접 Colab 로컬 스토리지로 다운로드합니다.
- **anima-preview.safetensors** — Transformer/DiT (~4.2GB)
- **qwen_3_06b_base.safetensors** — 텍스트 인코더 Qwen3-0.6B (~1.2GB)
- **qwen_image_vae.safetensors** — VAE (~0.2GB)

In [ ]:
#@title 2-1. Anima 모델 파일 다운로드
from huggingface_hub import hf_hub_download
import os

MODEL_DIR = "/content/models"
os.makedirs(MODEL_DIR, exist_ok=True)

REPO_ID = "circlestone-labs/Anima"

# Transformer (DiT) — anima-preview.safetensors
transformer_path = hf_hub_download(
    repo_id=REPO_ID,
    filename="split_files/diffusion_models/anima-preview.safetensors",
    local_dir=MODEL_DIR,
    local_dir_use_symlinks=False,
)
# 실제 저장 경로 정리
TRANSFORMER_PATH = os.path.join(MODEL_DIR, "split_files/diffusion_models/anima-preview.safetensors")
print(f"✅ Transformer: {TRANSFORMER_PATH}")
print(f"   크기: {os.path.getsize(TRANSFORMER_PATH)/1024**3:.2f} GB")

# Text Encoder — qwen_3_06b_base.safetensors
text_enc_path = hf_hub_download(
    repo_id=REPO_ID,
    filename="split_files/text_encoders/qwen_3_06b_base.safetensors",
    local_dir=MODEL_DIR,
    local_dir_use_symlinks=False,
)
LLM_PATH = os.path.join(MODEL_DIR, "split_files/text_encoders/qwen_3_06b_base.safetensors")
print(f"✅ Text Encoder: {LLM_PATH}")
print(f"   크기: {os.path.getsize(LLM_PATH)/1024**3:.2f} GB")

# VAE — qwen_image_vae.safetensors
vae_path = hf_hub_download(
    repo_id=REPO_ID,
    filename="split_files/vae/qwen_image_vae.safetensors",
    local_dir=MODEL_DIR,
    local_dir_use_symlinks=False,
)
VAE_PATH = os.path.join(MODEL_DIR, "split_files/vae/qwen_image_vae.safetensors")
print(f"✅ VAE: {VAE_PATH}")
print(f"   크기: {os.path.getsize(VAE_PATH)/1024**3:.2f} GB")

print(f"\n📦 모델 총 크기: {sum(os.path.getsize(p) for p in [TRANSFORMER_PATH, LLM_PATH, VAE_PATH])/1024**3:.2f} GB")

## 3. Google Drive 마운트 및 데이터셋 준비

데이터셋 구조: 이미지 파일 + 동일 이름의 `.txt` 캡션 파일  
캡션 형식: Danbooru 태그 (예: `shirakawa yuina, @artist, 1girl, solo, ...`)

In [ ]:
#@title 3-1. Google Drive 마운트 + 데이터셋 복사
from google.colab import drive
import shutil, glob

drive.mount('/content/drive')

# ========================================
# ⚠️ 여기를 자신의 Google Drive 경로로 수정하세요!
# ========================================
GDRIVE_DATASET_DIR = "/content/drive/MyDrive/yuina_anima"  #@param {type:"string"}
GDRIVE_OUTPUT_DIR  = "/content/drive/MyDrive/anima_training_output"  #@param {type:"string"}

# 로컬에 데이터셋 복사 (I/O 속도 향상)
LOCAL_DATASET_DIR = "/content/dataset"
if os.path.exists(LOCAL_DATASET_DIR):
    shutil.rmtree(LOCAL_DATASET_DIR)
shutil.copytree(GDRIVE_DATASET_DIR, LOCAL_DATASET_DIR)

# 데이터셋 확인
images = sorted(glob.glob(os.path.join(LOCAL_DATASET_DIR, "*.png")) +
                glob.glob(os.path.join(LOCAL_DATASET_DIR, "*.jpg")))
captions = sorted(glob.glob(os.path.join(LOCAL_DATASET_DIR, "*.txt")))
print(f"📁 데이터셋 경로: {LOCAL_DATASET_DIR}")
print(f"🖼️  이미지 수: {len(images)}")
print(f"📝 캡션 수:   {len(captions)}")

# 캡션 미리보기 (첫 3개)
print("\n--- 캡션 미리보기 ---")
for txt_file in captions[:3]:
    name = os.path.basename(txt_file)
    with open(txt_file, 'r', encoding='utf-8') as f:
        content = f.read().strip()
    print(f"  {name}: {content[:120]}{'...' if len(content)>120 else ''}")

# 출력 디렉토리 생성
os.makedirs(GDRIVE_OUTPUT_DIR, exist_ok=True)
print(f"\n💾 결과 저장 경로: {GDRIVE_OUTPUT_DIR}")

## 4. 학습 설정 파일 생성 (TOML)

사용자 조절 가능 파라미터는 `@param` 폼으로 제공됩니다.  
**고정값** (bf16, activation_checkpointing 등)은 T4에서 변경 시 NaN/OOM 위험이 있어 코드 내 고정.

> 💡 Colab에서 셀 왼쪽 폼을 통해 값을 변경할 수 있습니다.

In [ ]:
#@title 4-1. dataset.toml 생성
import os

# ========================================
# 데이터셋 설정 (@param으로 조절 가능)
# ========================================
RESOLUTION = 512           #@param [512, 768, 1024] {type:"raw"}
NUM_REPEATS = 2            #@param {type:"integer"}
CACHE_SHUFFLE_NUM = 1     #@param {type:"integer"}
MIN_AR = 0.5               #@param {type:"number"}
MAX_AR = 2.0               #@param {type:"number"}
NUM_AR_BUCKETS = 7         #@param {type:"integer"}

dataset_toml = f"""
# Anima LoRA 학습 데이터셋 설정

resolutions = [{RESOLUTION}]

# AR 버킷 설정
enable_ar_bucket = true
min_ar = {MIN_AR}
max_ar = {MAX_AR}
num_ar_buckets = {NUM_AR_BUCKETS}

# 이미지 전용
frame_buckets = [1]

[[directory]]
path = '{LOCAL_DATASET_DIR}'
num_repeats = {NUM_REPEATS}
cache_shuffle_num = {CACHE_SHUFFLE_NUM}
cache_shuffle_delimiter = ', '
""".strip()

dataset_toml_path = "/content/diffusion-pipe/anima_dataset.toml"
with open(dataset_toml_path, 'w') as f:
    f.write(dataset_toml)

print(f"✅ dataset.toml 생성: {dataset_toml_path}")
print(f"📐 해상도: {RESOLUTION}px | repeat: {NUM_REPEATS} | 셔플: {CACHE_SHUFFLE_NUM}")
print("---")
print(dataset_toml)

In [ ]:
#@title 4-2. 학습 설정 (training.toml) 생성
import os

# ========================================
# 학습 하이퍼파라미터 (@param으로 조절 가능)
# ========================================
EPOCHS = 100               #@param {type:"integer"}
LORA_RANK = 32             #@param {type:"integer"}
BLOCKS_TO_SWAP = 8        #@param {type:"integer"}
SAVE_EVERY_N_EPOCHS = 1   #@param {type:"integer"}
BATCH_SIZE = 10             #@param [1, 2, 4, 6, 8, 10, 12] {type:"raw"}
GRAD_ACCUM = 1             #@param [1, 2, 4] {type:"raw"}
WARMUP_STEPS = 100         #@param {type:"integer"}
CHECKPOINT_MINUTES = 30    #@param {type:"integer"}
GRADIENT_CLIPPING = 1.0    #@param {type:"number"}
LLM_ADAPTER_LR = 0        #@param {type:"number"}
TIMESTEP_SAMPLE = "logit_normal"  #@param ["logit_normal", "uniform", "sigmoid"] {type:"string"}

# ========================================
# Optimizer 설정
# ========================================
OPTIMIZER = "Prodigy"      #@param ["Prodigy", "AdamW8bitKahan", "adamw_optimi"] {type:"string"}
OPTIMIZER_LR = 1           #@param {type:"number"}
WEIGHT_DECAY = 0.01        #@param {type:"number"}

# Optimizer별 기본 안내
_optim_notes = {
    "Prodigy": "lr=1 고정 (자동 LR 조정), bf16 필수",
    "AdamW8bitKahan": "일반적으로 lr=5e-5~5e-6, 8bit 메모리 절약",
    "adamw_optimi": "일반적으로 lr=2e-5, diffusion-pipe example 기본값",
}
print(f"📋 Optimizer: {OPTIMIZER} — {_optim_notes.get(OPTIMIZER, '')}")

# === Optimizer TOML 블록 생성 ===
if OPTIMIZER == "Prodigy":
    optimizer_toml = f"""[optimizer]
type = 'Prodigy'
lr = {OPTIMIZER_LR}
betas = [0.9, 0.99]
weight_decay = {WEIGHT_DECAY}"""
elif OPTIMIZER == "AdamW8bitKahan":
    optimizer_toml = f"""[optimizer]
type = 'AdamW8bitKahan'
lr = {OPTIMIZER_LR}
betas = [0.9, 0.99]
weight_decay = {WEIGHT_DECAY}"""
else:
    optimizer_toml = f"""[optimizer]
type = '{OPTIMIZER}'
lr = {OPTIMIZER_LR}
betas = [0.9, 0.999]
weight_decay = {WEIGHT_DECAY}"""

training_toml = f"""
# Anima LoRA Training Config (Colab T4)
output_dir = '/content/training_output'
dataset = '/content/diffusion-pipe/anima_dataset.toml'

# ===== 학습 설정 =====
epochs = {EPOCHS}
micro_batch_size_per_gpu = {BATCH_SIZE}
pipeline_stages = 1
gradient_accumulation_steps = {GRAD_ACCUM}
gradient_clipping = {GRADIENT_CLIPPING}
warmup_steps = {WARMUP_STEPS}

# ===== Block Swap =====
blocks_to_swap = {BLOCKS_TO_SWAP}

# ===== Eval (비활성) =====
eval_every_n_epochs = {EPOCHS + 1}
eval_before_first_step = false
eval_micro_batch_size_per_gpu = 1
eval_gradient_accumulation_steps = 1

# ===== 저장 설정 =====
save_every_n_epochs = {SAVE_EVERY_N_EPOCHS}
checkpoint_every_n_minutes = {CHECKPOINT_MINUTES}
activation_checkpointing = true
partition_method = 'parameters'
save_dtype = 'bfloat16'
caching_batch_size = 1
steps_per_print = 1

# ===== 모델 설정 (bf16 고정 — fp16은 NaN 위험) =====
[model]
type = 'anima'
transformer_path = '{TRANSFORMER_PATH}'
vae_path = '{VAE_PATH}'
llm_path = '{LLM_PATH}'
dtype = 'bfloat16'
timestep_sample_method = '{TIMESTEP_SAMPLE}'
llm_adapter_lr = {LLM_ADAPTER_LR}

# ===== LoRA 설정 =====
[adapter]
type = 'lora'
rank = {LORA_RANK}
dtype = 'bfloat16'

# ===== Optimizer =====
{optimizer_toml}
""".strip()

training_toml_path = "/content/diffusion-pipe/anima_training.toml"
with open(training_toml_path, 'w') as f:
    f.write(training_toml)

eff_batch = BATCH_SIZE * GRAD_ACCUM
print(f"✅ training.toml 생성: {training_toml_path}")
print(f"📊 Effective batch: {BATCH_SIZE}×{GRAD_ACCUM}={eff_batch} | rank: {LORA_RANK} | swap: {BLOCKS_TO_SWAP}")
print(f"🔧 Optimizer: {OPTIMIZER} lr={OPTIMIZER_LR} | warmup: {WARMUP_STEPS} | clip: {GRADIENT_CLIPPING}")
print(f"💾 Save: every {SAVE_EVERY_N_EPOCHS}ep | checkpoint: {CHECKPOINT_MINUTES}min")
print("---")

## 5. TensorBoard 모니터링 (Cloudflare Tunnel)

학습 시작 **전에** TensorBoard를 먼저 띄워야 실시간 모니터링이 가능합니다.  
Colab에서는 localhost 접근이 불가하므로 Cloudflare Tunnel을 통해 외부에서 접속합니다.

> 셀 실행 후 출력되는 `🔗 LINK: https://xxx.trycloudflare.com` 링크를 클릭하세요.

In [ ]:
#@title 5-1. TensorBoard 실행 + Cloudflare Tunnel 연결
import os, subprocess, socket, threading, time

TB_PORT = 6006
TB_LOG_DIR = "/content/training_output"
os.makedirs(TB_LOG_DIR, exist_ok=True)

# --- Cloudflare Tunnel 설치 ---
if not os.path.exists("/usr/local/bin/cloudflared"):
    print("☁️  Cloudflare Tunnel 설치 중...")
    !wget -q -P /tmp/ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
    !dpkg -i /tmp/cloudflared-linux-amd64.deb
    print("✅ Cloudflare Tunnel 설치 완료")

# --- TensorBoard 백그라운드 시작 ---
!kill $(lsof -t -i:{TB_PORT}) 2>/dev/null; true

tb_proc = subprocess.Popen(
    ["tensorboard", "--logdir", TB_LOG_DIR, "--port", str(TB_PORT), "--bind_all"],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)
print(f"📊 TensorBoard 시작 (port {TB_PORT}, logdir: {TB_LOG_DIR})")

# --- TensorBoard 포트 대기 ---
print("⏳ TensorBoard 포트 대기 중...")
for _ in range(30):
    time.sleep(0.5)
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    result = sock.connect_ex(('127.0.0.1', TB_PORT))
    sock.close()
    if result == 0:
        break
print("✅ TensorBoard 준비 완료!")

# --- Cloudflare Tunnel 시작 + URL 동기적으로 대기 ---
print("☁️  Cloudflare Tunnel 연결 중...")
cf_proc = subprocess.Popen(
    ["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{TB_PORT}"],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)
# stderr에서 URL 나올 때까지 동기적으로 읽기 (ComfyUI 방식)
for line in cf_proc.stderr:
    l = line.decode()
    if "trycloudflare.com" in l:
        if "http" in l:
            print("🔗 LINK:", l[l.find("http"):].strip())
            print("   ↑ 이 링크를 클릭하면 실시간으로 loss를 확인할 수 있습니다")
            break

print("\n📌 링크 확인 후 다음 셀(학습 시작)을 실행하세요.")
print("   TensorBoard + Tunnel은 백그라운드로 계속 동작합니다.")

## 6. 학습 실행

DeepSpeed를 통해 diffusion-pipe 학습을 시작합니다.

**실행 흐름:**
1. Latent 캐싱 (VAE로 이미지 → latent 변환, 1회만)
2. Text embedding 캐싱 (Qwen3로 캡션 → embedding, 1회만)  
3. LoRA 학습 시작

> 📊 위에서 실행한 TensorBoard 링크에서 실시간 loss를 모니터링하세요!

In [ ]:
#@title 6-1. 학습 시작
import os
os.chdir('/content/diffusion-pipe')
os.environ['PYTORCH_ALLOC_CONF'] = 'expandable_segments:True'

!NCCL_P2P_DISABLE="1" NCCL_IB_DISABLE="1" \
    deepspeed --num_gpus=1 train.py \
    --deepspeed \
    --config anima_training.toml

In [ ]:
#@title 6-2. (선택) 체크포인트에서 이어 학습 (Colab 끊김 후)
# ============================================================
# 새 Colab 세션에서의 이어 학습 순서:
#   1. 셀 1~5 순서대로 실행 (환경 설정, 모델 다운로드, Drive 마운트, TOML 생성, TensorBoard)
#   2. 이 셀 실행 (Drive에서 체크포인트 복원 + 이어 학습)
# ============================================================

import os, shutil, glob, re

# --- Google Drive에서 체크포인트 복원 ---
OUTPUT_DIR = "/content/training_output"
GDRIVE_OUTPUT_DIR = "/content/drive/MyDrive/anima_training_output"  # 셀 3에서 설정한 경로와 동일하게!

def sort_by_step(path):
    """global_step 폴더를 숫자 기준으로 정렬 (global_step80 < global_step473)"""
    m = re.search(r'(\d+)$', os.path.basename(path))
    return int(m.group(1)) if m else 0

# Drive에 백업된 학습 결과가 있으면 로컬로 복원
if os.path.exists(GDRIVE_OUTPUT_DIR) and os.listdir(GDRIVE_OUTPUT_DIR):
    print("🔄 Google Drive에서 체크포인트 복원 중...")
    if os.path.exists(OUTPUT_DIR):
        shutil.rmtree(OUTPUT_DIR)
    shutil.copytree(GDRIVE_OUTPUT_DIR, OUTPUT_DIR)
    print(f"✅ 복원 완료: {GDRIVE_OUTPUT_DIR} → {OUTPUT_DIR}")

    # 가장 최근 run 자동 탐지 (diffusion-pipe와 동일 방식: sorted glob → 마지막)
    run_dirs = sorted(glob.glob(os.path.join(OUTPUT_DIR, "*")))
    if run_dirs:
        latest_run = run_dirs[-1]
        print(f"\n📂 최신 run: {os.path.basename(latest_run)}")
        for item in sorted(os.listdir(latest_run)):
            marker = "📁" if os.path.isdir(os.path.join(latest_run, item)) else "📄"
            print(f"   {marker} {item}")

        # global_step 체크포인트 확인 (숫자 기준 정렬!)
        ckpts = sorted(glob.glob(os.path.join(latest_run, "global_step*")), key=sort_by_step)
        if ckpts:
            latest_ckpt = os.path.basename(ckpts[-1])
            print(f"\n🔄 이어 학습 시작점: {latest_ckpt}")
        else:
            print("\n⚠️ DeepSpeed 체크포인트(global_step*)가 없습니다. 처음부터 학습됩니다.")

        # latest 파일 확인 (DeepSpeed가 실제로 읽는 파일)
        latest_file = os.path.join(latest_run, "latest")
        if os.path.exists(latest_file):
            with open(latest_file, 'r') as f:
                ds_tag = f.read().strip()
            print(f"   (DeepSpeed latest 파일: {ds_tag})")
else:
    print("⚠️ Google Drive에 백업된 체크포인트가 없습니다.")
    print(f"   경로 확인: {GDRIVE_OUTPUT_DIR}")

# --- 이어 학습 시작 ---
# --resume_from_checkpoint: diffusion-pipe가 자동으로 output_dir 내 최신 run → latest 파일 → 체크포인트 탐지
os.chdir('/content/diffusion-pipe')
os.environ['PYTORCH_ALLOC_CONF'] = 'expandable_segments:True'

!NCCL_P2P_DISABLE="1" NCCL_IB_DISABLE="1" \
    deepspeed --num_gpus=1 train.py \
    --deepspeed \
    --config anima_training.toml \
    --resume_from_checkpoint

## 7. 결과 확인 및 Google Drive로 내보내기

학습 완료 후 LoRA 파일을 Google Drive에 복사합니다.
- 저장 형식: **ComfyUI 호환** (`diffusion_model.` prefix 키 포함)
- ComfyUI에서 LoRA 노드로 바로 사용 가능

In [ ]:
#@title 7-1. 학습 결과 Google Drive 백업 (학습 중에도 실행 가능!)
# ⚠️ Colab 세션 종료 전에 반드시 실행하세요!
# 학습 중이라도 다른 셀에서 실행 가능합니다.
import glob, shutil, os, re

OUTPUT_DIR = "/content/training_output"
GDRIVE_OUTPUT_DIR = "/content/drive/MyDrive/anima_training_output"  # 셀 3에서 설정한 경로와 동일하게!

# 학습 결과 폴더 찾기 (가장 최신 run — diffusion-pipe와 동일 방식)
run_dirs = sorted(glob.glob(os.path.join(OUTPUT_DIR, "*")))
if not run_dirs:
    print("❌ 학습 결과를 찾을 수 없습니다.")
else:
    latest_run = run_dirs[-1]
    print(f"📂 최신 학습 run: {os.path.basename(latest_run)}\n")

    # 저장된 LoRA epoch 확인
    lora_dirs = sorted(glob.glob(os.path.join(latest_run, "epoch*")), key=lambda x: int(re.search(r'(\d+)', os.path.basename(x)).group(1)))
    print(f"📦 저장된 LoRA 체크포인트: {len(lora_dirs)}개")
    for d in lora_dirs:
        safetensors = os.path.join(d, "adapter_model.safetensors")
        if os.path.exists(safetensors):
            size_mb = os.path.getsize(safetensors) / 1024**2
            print(f"   {os.path.basename(d)}/adapter_model.safetensors ({size_mb:.1f} MB)")

    # DeepSpeed 체크포인트 확인 (이어 학습용 — global_step* 폴더)
    ckpt_dirs = sorted(glob.glob(os.path.join(latest_run, "global_step*")), key=lambda x: int(re.search(r'(\d+)', os.path.basename(x)).group(1)))
    if ckpt_dirs:
        total_ckpt_size = sum(
            sum(os.path.getsize(os.path.join(dp, f)) for f in fnames)
            for cd in ckpt_dirs for dp, _, fnames in os.walk(cd)
        ) / 1024**3
        print(f"\n🔄 DeepSpeed 체크포인트: {len(ckpt_dirs)}개 (이어 학습용, 총 {total_ckpt_size:.1f} GB)")
        for cd in ckpt_dirs:
            print(f"   📁 {os.path.basename(cd)}")
    else:
        print("\n⚠️ DeepSpeed 체크포인트 없음 (이어 학습 불가)")

    # TensorBoard 로그 확인
    tb_files = glob.glob(os.path.join(latest_run, "events.out.tfevents*"))
    if tb_files:
        print(f"\n📊 TensorBoard 로그: {len(tb_files)}개")

    # Google Drive로 복사 (run 폴더 전체 = LoRA + 체크포인트 + TB 로그 + TOML)
    print(f"\n🔄 Google Drive로 복사 중... → {GDRIVE_OUTPUT_DIR}")
    os.makedirs(GDRIVE_OUTPUT_DIR, exist_ok=True)
    dest = os.path.join(GDRIVE_OUTPUT_DIR, os.path.basename(latest_run))
    if os.path.exists(dest):
        shutil.rmtree(dest)
    shutil.copytree(latest_run, dest)

    # 복사 결과 요약
    dest_size = sum(
        os.path.getsize(os.path.join(dp, f))
        for dp, _, fnames in os.walk(dest) for f in fnames
    ) / 1024**3
    print(f"✅ 복사 완료: {dest} ({dest_size:.1f} GB)")
    print(f"   포함: LoRA({len(lora_dirs)}) + 체크포인트({len(ckpt_dirs)}) + TensorBoard + TOML")

    if lora_dirs:
        print(f"\n🎯 최종 LoRA: {os.path.basename(lora_dirs[-1])}/adapter_model.safetensors")
        print("   → ComfyUI의 loras/ 폴더에 넣어서 사용하세요")

---

## 📌 트러블슈팅 & 팁

### VRAM OOM이 발생하면
1. `BLOCKS_TO_SWAP`을 `14`~`18`로 올리기 (최대 26)
2. `resolutions`를 `[512]`로 낮추기
3. `activation_checkpointing = 'unsloth'`로 변경 (더 공격적 절약)

### 학습이 너무 느리면
- `blocks_to_swap`을 줄이기 (VRAM이 여유 있다면)
- `gradient_accumulation_steps`를 2로 낮추기

### 과적합 체크
- Loss가 초반에 빠르게 떨어진 후 더 이상 감소하지 않으면 과적합 가능성
- `save_every_n_epochs = 5`로 줄여서 중간 체크포인트 저장
- ComfyUI에서 각 epoch의 LoRA를 직접 눈으로 비교하는 것이 가장 확실

### 캡션 관련
- Anima 태그 순서: `[quality/meta] [1girl/1boy] [character] [series] [artist] [general tags]`
- 아티스트 태그는 반드시 `@` 접두사: `@artistname`
- 캡션 부족 시 빈 캡션으로 학습됨 (경고만 출력)

### Colab 세션 끊김 대비
- `checkpoint_every_n_minutes = 30`이 설정되어 있음
- 끊기면 셀 1~4 재실행 후 **5-2 셀 (이어 학습)** 실행